In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate

import imageio

import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis

In [ ]:
image_name = '00000B94.png'
image_path = pathlib.Path('.').resolve().parent.joinpath('data', 'images', 'lossless', image_name)

if not image_path.exists():
    url = f'https://zenodo.org/record/3520266/files/{image_name}?download=1'
    urllib.request.urlretrieve(url, image_path)

In [ ]:
degrees = 0
edge_lengths = [8, 12]
penumbra = 1
centre = [0, 0]

field = pymedphys._mocks.profiles.create_rectangular_field_function(centre, edge_lengths, penumbra, degrees)
x_profile = pymedphys._mocks.profiles.create_profile_function(
    0, edge_lengths[0], penumbra
)
y_profile = pymedphys._mocks.profiles.create_profile_function(
    0, edge_lengths[1], penumbra
)

In [ ]:
x = np.arange(-15, 30, 0.1)
y = np.arange(-15, 15, 0.1)

xx, yy = np.meshgrid(x, y)

zz = field(xx, yy)
plt.pcolormesh(xx, yy, zz)
plt.axis('equal')

In [ ]:
mock_initial_centre = pymedphys._wlutz.findfield.initial_centre(x, y, zz_simple)
mock_initial_centre

In [ ]:
interp_size = 20

x_len = 30
y_len = 40

field_x_interp = np.linspace(-interp_size/2, interp_size/2, x_len)
field_y_interp = np.linspace(-interp_size/2, interp_size/2, y_len)

In [ ]:
x_interp, y_interp = pymedphys._wlutz.createaxis.transform_axis(field_x_interp, field_y_interp, mock_initial_centre, degrees)

plt.plot(x_interp[0], x_interp[1], label='x-axis')
plt.plot(y_interp[0], y_interp[1], label='y-axis')
plt.pcolormesh(xx, yy, zz)

plt.legend()
plt.axis('equal')

In [ ]:
def define_penumbra_points_at_origin(edge_lengths):
    penumbra_range = np.linspace(-penumbra, penumbra, 11)
    
    def _each_edge(current_edge_length, orthogonal_edge_length):
        half_field_range = np.linspace(-orthogonal_edge_length/4, orthogonal_edge_length/4, 51)
        
        a_side_lookup = -current_edge_length/2 + penumbra_range
        b_side_lookup = current_edge_length/2 + penumbra_range
        current_axis_lookup = np.concatenate([a_side_lookup, b_side_lookup])
        
        return current_axis_lookup, half_field_range
    
    edge_points_left_right = _each_edge(edge_lengths[0], edge_lengths[1])
    edge_points_top_bot = _each_edge(edge_lengths[1], edge_lengths[0])
    
    xx_left_right, yy_left_right = np.meshgrid(*edge_points_left_right)
    xx_top_bot, yy_top_bot = np.meshgrid(*edge_points_top_bot[::-1])
    
    return xx_left_right, yy_left_right, xx_top_bot, yy_top_bot

In [ ]:
xx_left_right, yy_left_right, xx_top_bot, yy_top_bot = define_penumbra_points_at_origin(edge_lengths)

plt.pcolormesh(xx, yy, zz)
plt.scatter(xx_left_right, yy_left_right, s=1)
plt.scatter(xx_top_bot, yy_top_bot, s=1)

In [ ]:
penumbra_range = np.linspace(-penumbra, penumbra, 11)

x_penumbra_left_lookup = -edge_lengths[0]/2 + penumbra_range
y_penumbra_bot_lookup = -edge_lengths[1]/2 + penumbra_range

In [ ]:
plt.plot(x_penumbra_left_lookup, x_profile(x_penumbra_left_lookup))
plt.plot(x_penumbra_left_lookup, x_profile(-x_penumbra_left_lookup))

plt.plot(y_penumbra_bot_lookup, y_profile(y_penumbra_bot_lookup))
plt.plot(y_penumbra_bot_lookup, y_profile(-y_penumbra_bot_lookup))

In [ ]:
np.all(np.abs(np.diff(field(xx_left_right, yy_left_right), axis=0)) < 0.001)

In [ ]:
np.all(np.abs(np.diff(field(xx_top_bot, yy_top_bot), axis=1)) < 0.001)

In [ ]:
left_right = np.mean(field(xx_left_right, yy_left_right), axis=0)
top_bot = np.mean(field(xx_top_bot, yy_top_bot), axis=1)

In [ ]:
np.allclose(left_right, left_right[::-1])

In [ ]:
np.allclose(top_bot, top_bot[::-1])

In [ ]:
x_profile_penumbra = np.concatenate([x_profile(x_penumbra_left_lookup), x_profile(-x_penumbra_left_lookup[::-1])])
y_profile_penumbra = np.concatenate([y_profile(y_penumbra_bot_lookup), y_profile(-y_penumbra_bot_lookup[::-1])])

In [ ]:
np.allclose(left_right, x_profile_penumbra, rtol=0.001, atol=0.001)

In [ ]:
np.allclose(top_bot, y_profile_penumbra, rtol=0.001, atol=0.001)

In [ ]:
top_bot - y_profile_penumbra

In [ ]:
plt.plot(top_bot)
plt.plot(y_profile_penumbra)

In [ ]:
current_edge_length = edge_lengths[0]
orthogonal_edge_length = edge_lengths[1]

penumbra_range = np.linspace(-penumbra/2, penumbra/2, 11)
half_field_range = np.linspace(-orthogonal_edge_length/4, orthogonal_edge_length/4, 51)

a_side_lookup = -current_edge_length/2 + penumbra_range
b_side_lookup = current_edge_length/2 + penumbra_range

current_axis_lookup = np.concatenate([a_side_lookup, b_side_lookup])
curr_axis, orthog_axis = np.meshgrid(current_axis_lookup, half_field_range)

plt.pcolormesh(xx, yy, zz)
plt.scatter(curr_axis, orthog_axis)

In [ ]:
current_edge_length = edge_lengths[1]
orthogonal_edge_length = edge_lengths[0]

penumbra_range = np.linspace(-penumbra/2, penumbra/2, 11)
half_field_range = np.linspace(-orthogonal_edge_length/4, orthogonal_edge_length/4, 51)

a_side_lookup = -current_edge_length/2 + penumbra_range
b_side_lookup = current_edge_length/2 + penumbra_range

current_axis_lookup = np.concatenate([a_side_lookup, b_side_lookup])
curr_axis, orthog_axis = np.meshgrid(half_field_range, current_axis_lookup)

plt.pcolormesh(xx, yy, zz)
plt.scatter(curr_axis, orthog_axis)

In [ ]:
img = imageio.imread(image_path)

In [ ]:
assert img.dtype == np.dtype('uint16')
assert np.shape(img) == (1024, 1024)

In [ ]:
shape = np.shape(img)
x_axis = np.arange(-shape[1]/2, shape[1]/2)/4
y_axis = np.arange(-shape[0]/2, shape[0]/2)/4

In [ ]:
plt.imshow(img)
plt.xlim([400, 600])
plt.ylim([600, 400])
plt.colorbar()

In [ ]:
scale_and_flip = 1 - img[::-1,:] / 2**16
plt.contourf(x_axis, y_axis, scale_and_flip, 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
initial_centre = pymedphys._wlutz.findfield.initial_centre(x_axis, y_axis, scale_and_flip)
initial_centre

In [ ]:
interpolation = scipy.interpolate.RectBivariateSpline(x_axis, y_axis, scale_and_flip, kx=1, ky=1)

In [ ]:
scipy.ndimage.measurements.standard_deviation(scale_and_flip)